# Creación de una Base de Datos Analítica

## Integrantes

#### Raul Alberto Niño Cuervo
#### Jhoan Sebastian Zamudio

## Problemática Abordada

#### La empresa AutoMercado de compra y venta de vehículos usados necesita contar con una base de datos que centralice la información de los carros disponibles para la venta. Actualmente los datos se encuentran dispersos y sin estructura, lo que dificulta comparar precios, identificar disponibilidad, conocer el historial del vehículo y analizar las preferencias del mercado.
#### Al organizar los datos en una base estructurada, la empresa podrá consultar rápidamente vehículos según características específicas (año, transmisión, combustible, precio), evaluar tendencias de demanda y tomar decisiones de compra y venta basadas en datos, mejorando así la eficiencia comercial y la competitividad en el mercado.

## Objetivo

#### Desarrollar una base de datos estructurada para AutoMercado Select que permita almacenar, consultar y analizar la información de los vehículos usados en inventario, facilitando la comparación de precios, el seguimiento de disponibilidad y el soporte a decisiones comerciales basadas en datos

## Dataset Escogido

Nombre: Datos de coches usados ​​en sitios web

Fuente: kaggle

Enlace: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv

Autor: Nehal Birla · Nishant Verma · Nikhil Kushwaha

Escogimos este dataset porque contiene información real del mercado de vehículos usados, permitiendo analizar cómo influyen factores como la marca, modelo, año, kilometraje, tipo de combustible y transmisión en el precio de venta. Además, ofrece datos variados y suficientemente completos para construir un modelo entidad-relación, normalizar la información y realizar consultas analíticas que apoyen la toma de decisiones en un contexto de compra y venta de autos.

### Variables Relevantes

| **Variable (ES)** | **Variable (EN)** | **Tipo**   | **Descripción**                                                  |
| ----------------- | ----------------- | ---------- | ---------------------------------------------------------------- |
| **Make**          | Brand             | Categórica | Marca del vehículo.                                              |
| **Model**         | Model             | Categórica | Modelo específico del vehículo.                                  |
| **Year**          | Year              | Numérica   | Año de fabricación del vehículo.                                 |
| **Price**         | Price             | Numérica   | Precio de venta actual del vehículo.                             |
| **Kilometer**     | Mileage           | Numérica   | Kilometraje recorrido por el vehículo (nivel de uso y desgaste). |
| **Fuel_Type**     | Fuel Type         | Categórica | Tipo de combustible utilizado.                                   |
| **Transmission**  | Transmission      | Categórica | Tipo de transmisión (Manual o Automática).                       |
| **Owner**         | Previous Owners   | Categórica | Cantidad de dueños anteriores.                                   |
| **Seller_Type**   | Seller Type       | Categórica | Tipo de vendedor (Particular o Concesionario).                   |


### Modelo Entidad-Relación

El modelo entidad relacion es el archivo ERD de la carpeta actividad 1

### Cargar el CSV

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


### Importación de Librerias

In [0]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

import matplotlib.pyplot as plt
import seaborn as sns

import kagglehub

### Funciones para Descargar, Extraer y el Leer el Dataset

In [0]:
def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path

def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir):
        #os.makedirs('src/static/csv', exist_ok=True)
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")

        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            try:
                df = pd.read_csv(file_path, encoding="latin1")
            except Exception as e:
                print(f"Error al leer {file}: {e}")
                continue
            print(f"Creando/actualizando ")
        print("cvs creado correctamente en ")
        return df

### Descarga del Dataset

In [0]:
# 1. Descargar el dataset desde Kaggle
dataset_path = download_dataset_zip("nehalbirla/vehicle-dataset-from-cardekho")

# 2. Extraer los archivos
csv_dir = extract_zip_files(dataset_path)

# 3. Crear el DataFrame de Pandas
df_pd = create_csv(csv_dir)

# 4. Mostrar las primeras filas
df_pd.head(4)


Descargando dataset desde Kaggle...
Ruta al dataset: /home/spark-58c27540-13ab-41d5-a7bd-f2/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-58c27540-13ab-41d5-a7bd-f2/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4/car details v4.csv...
Creando/actualizando 
Leyendo /home/spark-58c27540-13ab-41d5-a7bd-f2/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4/car data.csv...
Creando/actualizando 
Leyendo /home/spark-58c27540-13ab-41d5-a7bd-f2/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4/Car details v3.csv...
Creando/actualizando 
Leyendo /home/spark-58c27540-13ab-41d5-a7bd-f2/.cache/kagglehub/datasets/nehalbirla/vehicle-dataset-from-cardekho/versions/4/CAR DETAILS FROM CAR DEKHO.csv...
Creando/actualizando 
cvs creado correctamente en 


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner


### Convertir Dataframe de pandas a Spark

In [0]:
df = spark.createDataFrame(df_pd)

### Creación la Tabla e Inserto de Datos

In [0]:
df = df.toDF(*[c.replace(" ", "_") for c in df.columns])

df.write.mode("overwrite").saveAsTable("tbl_ventas_carros")

### Verifica la creación correcta de la tabla

In [0]:
%sql
SELECT * 
FROM tbl_ventas_carros
LIMIT 5;
     

name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
Hyundai i10 Magna 1.1,2008,150000,90000,Petrol,Individual,Manual,Third Owner
Hyundai Santro Xing GLS,2009,110000,120000,Petrol,Individual,Manual,First Owner
Nissan Sunny Diesel XV,2013,450000,80000,Diesel,Individual,Manual,First Owner
Hyundai i20 1.2 Spotz,2017,400000,38000,Petrol,Individual,Manual,Second Owner
Hyundai Santro Xing GLS,2009,140000,120000,Petrol,Individual,Manual,First Owner


### Conteo de Registros

In [0]:
%sql
SELECT COUNT(*) FROM tbl_ventas_carros;

COUNT(*)
4340


### Nombres y Tipos de Columnas

In [0]:

%sql
DESCRIBE TABLE tbl_ventas_carros;

col_name,data_type,comment
name,string,null
year,bigint,null
selling_price,bigint,null
km_driven,bigint,null
fuel,string,null
seller_type,string,null
transmission,string,null
owner,string,null


Esta consulta muestra la estructura de la tabla tbl_ventas_carros, es decir, lista cada columna junto con su tipo de dato y si permite valores nulos. Sirve para conocer cómo está organizada la información dentro de la tabla.

### Consulta con Filtro

In [0]:
%sql
SELECT name,  Year, selling_price, fuel
FROM tbl_ventas_carros
WHERE fuel = 'Petrol'
ORDER BY selling_price ASC
LIMIT 10;



name,Year,selling_price,fuel
Ford Ikon 1.6 ZXI NXt,2005,20000,Petrol
Ford Ikon 1.4 ZXi,2000,22000,Petrol
Maruti 800 EX,2004,30000,Petrol
Tata Nano Std BSII,2009,35000,Petrol
Tata Nano LX SE,2012,35000,Petrol
OpelCorsa 1.4 GL,2002,35000,Petrol
Maruti 800 Std,2004,37500,Petrol
Tata Nano Std,2011,40000,Petrol
Maruti 800 Std,1998,40000,Petrol
Maruti 800 EX,2001,40000,Petrol


Esta consulta selecciona los vehículos que funcionan con gasolina (Fuel_Type = 'Petrol') y muestra sus datos básicos: marca, modelo, año y precio. Luego los ordena de menor a mayor precio (ORDER BY Price ASC) y devuelve solo los 10 vehículos más económicos (LIMIT 10).

### Creacion de la base de datos

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS automercado_select;
USE automercado_select;

CREATE TABLE IF NOT EXISTS marcas (
    id_marca BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    nombre_marca STRING NOT NULL
);

CREATE TABLE IF NOT EXISTS modelos (
    id_modelo BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    nombre_modelo STRING NOT NULL,
    id_marca BIGINT NOT NULL,
    FOREIGN KEY (id_marca) REFERENCES marcas(id_marca)
);

CREATE TABLE IF NOT EXISTS combustibles (
    id_combustible BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    tipo_combustible STRING NOT NULL
);

CREATE TABLE IF NOT EXISTS transmisiones (
    id_transmision BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    tipo_transmision STRING NOT NULL
);

CREATE TABLE IF NOT EXISTS vendedores (
    id_vendedor BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    tipo_vendedor STRING NOT NULL  
);

CREATE TABLE IF NOT EXISTS carros (
    id_carro BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    anio INT,
    kilometraje BIGINT,
    color STRING,
    precio BIGINT,
    motor STRING,
    potencia_max STRING,
    torque_max STRING,
    capacidad_tanque DOUBLE,
    asientos DOUBLE,
    largo DOUBLE,
    ancho DOUBLE,
    alto DOUBLE,
    id_modelo BIGINT NOT NULL,
    id_combustible BIGINT NOT NULL,
    id_transmision BIGINT NOT NULL,
    id_vendedor BIGINT NOT NULL,
    FOREIGN KEY (id_modelo) REFERENCES modelos(id_modelo),
    FOREIGN KEY (id_combustible) REFERENCES combustibles(id_combustible),
    FOREIGN KEY (id_transmision) REFERENCES transmisiones(id_transmision),
    FOREIGN KEY (id_vendedor) REFERENCES vendedores(id_vendedor)
);


La base de datos AutoMercado Select permite organizar y administrar de manera estructurada la información relacionada con la venta de vehículos usados. A través de su modelo entidad–relación, se logra separar adecuadamente los datos del fabricante, modelos, características y transacciones, lo cual facilita el análisis posterior. Con esta estructura, es posible consultar tendencias de precios, preferencias de combustible, variaciones entre marcas y el comportamiento del mercado en general. La organización clara y normalizada de los datos contribuye a mejorar la calidad de las decisiones comerciales, permitiendo identificar patrones relevantes y apoyar estrategias de venta y adquisición de vehículos dentro del negocio.